# AFAP Cash Flow Engine — Validation Notebook

This notebook validates the **AFAP Phase 3 Cash Flow Health Engine**.

The purpose of this notebook is to:
1. Confirm deterministic execution of `cash_flow_engine`
2. Validate that outputs conform to AFAP Phase 3 schema requirements
3. Inspect representative engine outputs for interpretability
4. Ensure internal schema validation is enforced

This notebook is part of **AFAP Phase 3 — Locked Engines**.


## Environment Setup

This cell configures the Python environment so the AFAP project modules  
can be imported correctly when running the notebook from the `/notebooks` directory.

We explicitly append the project root to `sys.path` to enable absolute imports
from the `engines` package.


In [5]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))


## Import Cash Flow Engine

We import the **Cash Flow Health Engine**, which:
- Computes operating profit
- Derives a financing coverage proxy
- Applies deterministic severity rules
- Performs internal schema validation before returning results


In [6]:
from engines.cash_flow_engine import cash_flow_engine


## Load Financial Statement Data

This dataset contains cleaned, standardized financial statement data
used across all AFAP engines.

Expected columns include:
- Company
- Year
- FS Category
- FS Subcategory
- Amount

The Cash Flow Engine relies on this structure to compute operating performance
and financing pressure indicators.


In [7]:
import pandas as pd

financials = pd.read_csv("../data/cleaned/financial_statements.csv")
financials.head()


,Company,Year,FS Category,FS Subcategory,Statement,Amount
0,Acme Manufacturing Ltd,2020,Assets,Current Assets,Balance Sheet,3109667
1,Acme Manufacturing Ltd,2020,Assets,Non-Current Assets,Balance Sheet,905812
2,Acme Manufacturing Ltd,2020,Equity,Equity,Balance Sheet,2152630
3,Acme Manufacturing Ltd,2020,Expenses,COGS,Income Statement,373114
4,Acme Manufacturing Ltd,2020,Expenses,Finance Costs,Income Statement,2929304


## Execute Cash Flow Engine

We now run the Cash Flow Engine against the full financials dataset.

Internally, the engine:
- Groups data by Company and Year
- Computes operating profit
- Calculates a coverage proxy
- Assigns a top-level severity
- Generates a human-readable explanation
- Validates output structure via `validate_engine_output`


In [8]:
cash_flow_results = cash_flow_engine(financials)


✅ cash_flow_engine output validated successfully.


## Inspect Engine Output

We inspect the first few results to confirm:

- Engine metadata is present
- Metrics are correctly computed
- Flags are deterministic and boolean
- Severity is top-level (not embedded in flags)
- Explanation text aligns with severity logic

This inspection confirms readiness for orchestration.


In [9]:
cash_flow_results[:2]


[{'engine': 'cash_flow_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': np.int64(2020),
  'metrics': {'operating_profit': np.int64(813322),
   'coverage_proxy': np.float64(0.2776502541217982)},
  'flags': {'negative_operating_profit': np.False_, 'weak_coverage': np.True_},
  'severity': 'watch',
  'explanation': 'Cash generation shows signs of pressure and warrants monitoring.'},
 {'engine': 'cash_flow_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': np.int64(2021),
  'metrics': {'operating_profit': np.int64(920171),
   'coverage_proxy': np.float64(0.38882463675949536)},
  'flags': {'negative_operating_profit': np.False_, 'weak_coverage': np.True_},
  'severity': 'watch',
  'explanation': 'Cash generation shows signs of pressure and warrants monitoring.'}]

## Validation Notes

✔ Output schema validation is enforced **inside the engine**  
✔ Severity is emitted as a top-level field  
✔ Metrics are numeric or null-safe  
✔ Flags are deterministic and auditable  
✔ Explanations are non-prescriptive and consistent  

This confirms that `cash_flow_engine` meets **AFAP Phase 3 compliance** and
is safe for orchestration and downstream AI interpretation.
